In [1]:
pip install transformers

In [2]:
import pandas as pd
from transformers import BertTokenizer, BertForPreTraining
import torch

# Load your drug dataset
df = pd.read_csv('new DS.csv')

# Display the first few rows of the dataset
print(df.head())

   Pka   Molecular Weight  Drug Affinity   Solubility              Name  \
0  8.5              300.5           High           50           Aspirin   
1  7.0              450.2       Moderate           30         Ibuprofen   
2  9.2              600.8            Low           20   Diphenhydramine   
3  6.8              700.3           High           40         Metformin   
4  5.5              550.6      Very High           25          Warfarin   

   Side Effects                          Use  \
0      Headache                  Pain relief   
1        Nausea       Inflammation reduction   
2    Drowsiness               Allergy relief   
3      Diarrhea   Type 2 diabetes management   
4      Bleeding              Anticoagulation   

                      Mechanism of Action                       Dosage  \
0   Inhibition of prostaglandin synthesis       500 mg every 4-6 hours   
1                   COX enzyme inhibition   200-400 mg every 4-6 hours   
2             Histamine receptor block

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
text_data = df[[' Drug Affinity',' Name', ' Side Effects', ' Use', ' Mechanism of Action', ' Contraindications']].astype(str)
tokenized_text = tokenizer(text_data.values.flatten().tolist(), return_tensors='pt', padding=True, truncation=True)

# Normalize numerical features
numerical_data = df[['Pka', ' Molecular Weight', ' Solubility']]
numerical_data = (numerical_data - numerical_data.mean()) / numerical_data.std()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
input_ids = tokenized_text['input_ids']
attention_mask = tokenized_text['attention_mask']
numerical_input = torch.tensor(numerical_data.values, dtype=torch.float32)

# Concatenate tokenized text and numerical input
model_input = {'input_ids': input_ids, 'attention_mask': attention_mask, 'numerical_input': numerical_input}

In [ ]:
# Example: Using a custom model that takes both textual and numerical inputs
class CustomModel(BertForPreTraining):
    def forward(self, input_ids=None, attention_mask=None, numerical_input=None):
        outputs = super().forward(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs['pooler_output']
        combined_output = torch.cat([pooled_output, numerical_input], dim=1)
        # Add your custom layers or predictions here
        return combined_output

model = CustomModel.from_pretrained('bert-base-uncased')
outputs = model(**model_input)

In [ ]:
model.save_pretrained('pretrained_drug_model')